**Imports**

In [20]:
import subprocess
import json
from IPython.display import Video

**Resolution Modification**

In [5]:
def change_resolution(input_video, output_video, width, height):
    ffmpeg_cmd = [
        'ffmpeg',
        '-i', input_video,
        '-vf', f'scale={width}:{height}',
        '-c:v', 'libx264',
        '-crf', '23',
        '-c:a', 'copy',
        output_video
    ]
    
    subprocess.run(ffmpeg_cmd)
    print(f"Video resized to {width}x{height}")

In [ ]:
input_video = 'input.mov'
output_video = 'output.mov'
width = 320
height = 180
change_resolution(input_video, output_video, width, height)

**Modify Chroma Subsampling**

In [14]:
def change_chroma_subsampling(input_video, output_video, pix_fmt):
    ffmpeg_cmd = [
        'ffmpeg',
        '-i', input_video,
        '-vf', 'pad=width=ceil(iw/2)*2:height=ceil(ih/2)*2', #My file does not have even pixels, so I pad it
        '-pix_fmt', pix_fmt,
        '-c:v', 'libx264',
        '-crf', '23',
        '-c:a', 'copy',
        output_video
    ]
    subprocess.run(ffmpeg_cmd)
    print(f"Chroma subsampling changed to {pix_fmt}")


In [ ]:
input_video = 'input.mov'
format = 'yuv444p'

output_video = 'output.mov'
change_chroma_subsampling(input_video, output_video, format)


**Read Video and print features**

In [21]:
def get_video_features(input_video):
    cmd = [
        'ffprobe',
        '-v', 'quiet',
        '-print_format', 'json',
        '-show_format',
        '-show_streams',
        input_video
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    video_info = json.loads(result.stdout)

    video_stream = None
    audio_stream = None

    for stream in video_info['streams']:
        if stream['codec_type'] == 'video':
            video_stream = stream
        elif stream['codec_type'] == 'audio':
            audio_stream = stream

    format_info = video_info['format']

    print("Video Information:")
    print("-----------------")
    print(f"Duration: {float(format_info['duration']):.2f} seconds")
    print(f"Resolution: {video_stream['width']}x{video_stream['height']}")
    print(f"Codec: {video_stream['codec_name']}")
    print(f"Bitrate: {int(format_info['bit_rate'])/1000:.2f} kb/s")
    print(f"Frame rate: {eval(video_stream['r_frame_rate']):.2f} fps")

In [ ]:
video_file = 'input.mov'
info = get_video_features(video_file)


**BBB container**

In [28]:
def create_bbb_container(input_video, output_video, length = 20):
    #cut the video
    cut_video = f'{output_video}_cut.mp4'
    subprocess.run([
        'ffmpeg',
        '-i', input_video,
        '-t', str(length),
        '-c:v', 'copy',
        '-c:a', 'copy',
        cut_video
    ])

    #export audio mono
    audio_file_aac = 'audio_mono.aac'
    subprocess.run([
        'ffmpeg',
        '-i', cut_video,
        '-vn',
        '-c:a', 'aac',
        '-ac', '1',
        '-b:a', '128k',
        audio_file_aac
    ])

    #export audio stereo
    audio_file_mp3 = 'audio_stereo.mp3'
    subprocess.run([
        'ffmpeg',
        '-i', cut_video,
        '-vn',
        '-c:a', 'libmp3lame',
        '-ac', '2',
        '-b:a', '96k',
        audio_file_mp3
    ])

    #export audio ac3
    ac3_audio = 'audio_ac3.ac3'
    subprocess.run([
        'ffmpeg',
        '-i', cut_video,
        '-vn',
        '-c:a', 'ac3',
        '-b:a', '192k',
        ac3_audio
    ])

    #pack everything
    subprocess.run([
        'ffmpeg',
        '-i', cut_video,
        '-i', audio_file_aac,
        '-i', audio_file_mp3,
        '-i', ac3_audio,
        '-map', '0:v',
        '-map', '1:a',
        '-map', '2:a',
        '-map', '3:a',
        '-c:v', 'copy',
        '-c:a', 'copy',
        output_video
    ])


In [ ]:
input_video = 'input.mov'
output_video = 'output.mp4'
create_bbb_container(input_video, output_video, length = 20)

**Track reader**


In [30]:
def track_reader(input_video):
    cmd = [
        'ffprobe',
        '-v', 'quiet',
        '-print_format', 'json',
        '-show_streams',
        input_video
    ]

    result = subprocess.run(cmd, capture_output=True, text=True)
    track_info = json.loads(result.stdout)

    video_tracks = 0
    audio_tracks = 0

    for stream in track_info['streams']:
        codec_type = stream['codec_type']
        if codec_type == 'video':
            video_tracks += 1
        elif codec_type == 'audio':
            audio_tracks += 1
    return video_tracks, audio_tracks


In [ ]:
input_video = 'output.mp4'
video, audio = track_reader(input_video)
print(f"Video tracks: {video}")
print(f"Audio tracks: {audio}")

**Show macroblocks and motion vectors**

In [42]:
def macroblocks_and_motion_vectors(input_video, output_video):
    debug_filter = 'codecview=mv=1:mvt=1:qp=1'
    cmd = [
        'ffmpeg',
        '-flags2', '+export_mvs',
        '-i', input_video,
    '-vf', 'pad=ceil(iw/16)*16:ceil(ih/16)*16,codecview=mv=1:mvt=2:qp=1',
        '-c:v', 'libx264',
        output_video
    ]

    result = subprocess.run(cmd, capture_output=True, text=True)

In [43]:
input_video = 'input.mov'
output_video = 'output.mp4'
macroblocks_and_motion_vectors(input_video, output_video)

**YUV**

In [46]:
def yuv(input_video, output_video):
    cmd = [
        'ffmpeg',
        '-i', input_video,
        '-vf', 'pad=ceil(iw/2)*2:ceil(ih/2)*2,split=2[a][b];[b]histogram=display_mode=overlay:level_height=244[histo];[a][histo]overlay=x=W-w:y=0',
        '-c:v', 'libx264',
        output_video
    ]

    subprocess.run(cmd)



In [ ]:
input_video = 'input.mov'
output_video = 'output.mp4'
yuv(input_video, output_video)